# JL124 E6C 6-circle example

Compare with data acquired using SPEC

In _hkl_ *E6C* geometry (https://people.debian.org/~picca/hkl/hkl.html#orge5e0490):

<img src="4S+2D.png" alt="E6C geometry" width="30%"/>

* xrays incident on the $\vec{x}$ direction (1, 0, 0)

axis  | moves    | rotation about axis
---   | ---      | ---
mu    | sample   | $\vec{z}$ `[0 0 1]`
omega | sample   | $-\vec{y}$ `[0 -1 0]`
chi   | sample   | $\vec{x}$ `[1 0 0]`
phi   | sample   | $-\vec{y}$ `[0 -1 0]`
gamma | detector | $\vec{z}$ `[0 0 1]`
delta | detector | $-\vec{y}$ `[0 -1 0]`

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spec2nexus

import gi
gi.require_version('Hkl', '5.0')
from hkl.diffract import E6C
from hkl.util import Lattice

from ophyd import (PseudoSingle, SoftPositioner)
from ophyd import Component as Cpt

In SPEC *sixc* geometry (https://certif.com/spec_help/sixc.html):

name    | mnemonic   | description
-----   | -----      | -----
Delta   | del        | Detector arm rotation
Theta   | th         | Rotates sample circles
Chi     | chi        | Sample tilt
Phi     | phi        | Sample rotation
Mu      | mu         | Diffractometer rotation
Gamma   | gam        | Out-of-plane detector rotation

> When the azimuthal angle is set to 90 degrees in the azimuth-fixed modes (3, 6, 9 and 12), the incident angle alpha will be equal to the exit angle beta.

In [20]:
# mapping of axis names between E6C and SPEC
SPEC_AXIS_NAMES = {
    # remap so we can use the sixc names
    # E6C  sixc
    'mu': 'mu',  # Diffractometer rotation around vertical axis
    'omega': 'theta',  # Rotates chi around horizontal axis
    'chi': 'chi',  # Rotates phi around beam axis
    'phi': 'phi',  # Sample rotation around horizontal axis (when phi is co-linear with omega)
    'gamma': 'gamma',  # Diffractometer rotation around vertical axis
    'delta': 'delta',  # Detector arm rotation
    }

In [23]:
class Diffractometer(E6C):
    h = Cpt(PseudoSingle, '')
    k = Cpt(PseudoSingle, '')
    l = Cpt(PseudoSingle, '')

    # use the SPEC axis names here
    mu = Cpt(SoftPositioner)
    theta = Cpt(SoftPositioner)
    chi = Cpt(SoftPositioner)
    phi = Cpt(SoftPositioner)
    gamma = Cpt(SoftPositioner)
    delta = Cpt(SoftPositioner)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        for p in self.real_positioners:
            p._set_position(0)  # give each a starting position

In [22]:
sixc = Diffractometer("", name="sixc")

sixc.calc.physical_axis_names = SPEC_AXIS_NAMES

```
sample: JL124_1
crystal:  3.905 3.905 3.905 90 90 90
geometry: sixc
mode: 12 (Z-Axis with Azimuth fixed and Chi, Phi set to -Sigma, -Tau)
lambda: 0.8265616267
r1: (0, 0, 2) 0.003 90 0.5799999712 239.9999477 12.102 12.9945
r2: (3, 0, 3) 47.18 90 0.5799999712 239.9999477 21.77425 15.7375
Q: (2.99804, 0.00216068, 2.99661) 47.14125 90.089 0.58 239.94275 21.73025 15.7375
UB: 1.207702707 1.248454819 0.002095582696 
    -1.485612421 0.9118074731 0.003241829804 
    -0.0173752388 0.02282507942 1.651530555
```

In [15]:
lattice = Lattice(
    a=3.905, b=3.905, c=3.905,
    alpha=90.0, beta=90.0, gamma=90.0)

# add the sample to the calculation engine
sixc.calc.new_sample(
    "JL124_1",
    lattice=Lattice(
        a=3.905, b=3.905, c=3.905,
        alpha=90.0, beta=90.0, gamma=90.0)
    )

HklSample(name='JL124_1', lattice=LatticeTuple(a=3.905, b=3.905, c=3.905, alpha=90.0, beta=90.0, gamma=90.0), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), U=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), UB=array([[ 1.60901032e+00, -9.85234670e-17, -9.85234670e-17],
       [ 0.00000000e+00,  1.60901032e+00, -9.85234670e-17],
       [ 0.00000000e+00,  0.00000000e+00,  1.60901032e+00]]), reflections=[])

In [16]:
sixc.calc.wavelength = 0.8265616267 # angstrom

# SPEC motors (in order): del th chi phi mu gam
# r1: (0, 0, 2) 0.003 90 0.5799999712 239.9999477 12.102 12.9945
# r2: (3, 0, 3) 47.18 90 0.5799999712 239.9999477 21.77425 15.7375

r1 = sixc.calc.sample.add_reflection(
    0, 0, 2, 
    position=sixc.calc.Position(
        delta=0.003, 
        theta=90, 
        chi=0.5799999712,
        phi=239.9999477, 
        mu=12.102, 
        gamma=12.9945,
        )
    )
r2 = sixc.calc.sample.add_reflection(
    3, 0, 3, 
    position=sixc.calc.Position(
        delta=47.18, 
        theta=90, 
        chi=0.5799999712,
        phi=239.9999477, 
        mu=21.77425, 
        gamma=15.7375,
        )
    )
sixc.calc.sample.compute_UB(r1, r2)

1

In [1]:
# UB: 1.207702707 1.248454819 0.002095582696 
#     -1.485612421 0.9118074731 0.003241829804 
#     -0.0173752388 0.02282507942 1.651530555

# FIXME:  must get same numbers, probably in different rows

sixc.UB.get()

NameError: name 'sixc' is not defined

In [6]:
print('calc.wavelength is', sixc.calc.wavelength)
print('sample is', sixc.calc.sample)
print('position is', sixc.position)

print('sample name is', sixc.sample_name.get())
print('u matrix is', sixc.U.get(), sixc.U.describe())
print('ub matrix is', sixc.UB.get(), sixc.UB.describe())
print('reflections:',
        sixc.reflections.get(),
        sixc.reflections.describe())
print('ux is', sixc.ux.get(), sixc.ux.describe())
print('uy is', sixc.uy.get(), sixc.uy.describe())
print('uz is', sixc.uz.get(), sixc.uz.describe())
print('lattice is', sixc.lattice.get(), sixc.lattice.describe())
print(sixc.read())

calc.wavelength is 0.8265616267
sample is HklSample(name='main', lattice=LatticeTuple(a=1.54, b=1.54, c=1.54, alpha=90.0, beta=90.0, gamma=90.0), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=-158.00938111589508, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=-70.82628430601865, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=-135.46551192711627, fit=True, inverted=False, units='Degree'), U=array([[-0.23411665,  0.23034296, -0.94452714],
       [ 0.39819763,  0.909018  ,  0.12298345],
       [ 0.88692054, -0.34731599, -0.30453827]]), UB=array([[-0.95519369,  0.93979709, -3.85366171],
       [ 1.62464252,  3.70878476,  0.5017713 ],
       [ 3.61862734, -1.41704593, -1.24251323]]), reflections=[(h=0.0, k=0.0, l=2.0), (h=3.0, k=0.0, l=3.0)], reflection_measured_angles=array([[0.        , 1.34386914],
   